# Importing Libraries

In [3]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from Mat_Package import Grapher

# Make Giant-Graph

In [4]:
lab_files_path = "./New-Dataset/labels/"
files = os.listdir(lab_files_path)

df = pd.read_csv(lab_files_path+files[0])
G = Grapher.makeGraph(df)
print("Iteration 0")
print("= = = = = = = = = =")

for i in range(1, 100):
    print("Iteration ",i)
    print("= = = = = = = = = =")
    df1 = pd.read_csv(lab_files_path+files[i])
    I_Graph = Grapher.makeGraph(df1)
    G = nx.compose(G, I_Graph)
    


Iteration 0
= = = = = = = = = =
Iteration  1
= = = = = = = = = =
Iteration  2
= = = = = = = = = =
Iteration  3
= = = = = = = = = =
Iteration  4
= = = = = = = = = =
Iteration  5
= = = = = = = = = =
Iteration  6
= = = = = = = = = =
Iteration  7
= = = = = = = = = =
Iteration  8
= = = = = = = = = =
Iteration  9
= = = = = = = = = =
Iteration  10
= = = = = = = = = =
Iteration  11
= = = = = = = = = =
Iteration  12
= = = = = = = = = =
Iteration  13
= = = = = = = = = =
Iteration  14
= = = = = = = = = =
Iteration  15
= = = = = = = = = =
Iteration  16
= = = = = = = = = =
Iteration  17
= = = = = = = = = =
Iteration  18
= = = = = = = = = =
Iteration  19
= = = = = = = = = =
Iteration  20
= = = = = = = = = =
Iteration  21
= = = = = = = = = =
Iteration  22
= = = = = = = = = =


# Printing Graph Info

In [ ]:
print("No. of Edges in graph are: ", G.number_of_edges())
print("No. of Nodes in graph are: ", G.number_of_nodes())

No. of Edges in graph are:  15543
No. of Nodes in graph are:  3318


# Random-Walk

In [ ]:
import random
random.seed(666)

def random_walk(G, start=None, path_length=100, alpha=0, rand=random.Random()):
    '''return a random walk path'''
    if start:
        path = [start]
    else:
        path = [rand.choice(list(G.nodes()))]
    while len(path) < path_length:
        cur = path[-1]
        #find it's neighbors
        if len(G[cur]) > 0:
            if rand.random() >= alpha:
                path.append(rand.choice(list(nx.all_neighbors(G, cur))))
            else:
                path.append(path[0])
        else:
            break
    return path


print(random_walk(G, start='BHD'))


['BHD', '2o0f3', 'Expected', 'S120043446', 'lofl', 'Page', 'S1z0040554', 'IBC', 'S1z0040554', 'Page', '?tal', '3,205.17', 'Cost', '1.41', '10', '1.58', '2.15', '40', 'RED', '-']


In [ ]:
def build_deepwalk_corpus(G, num_paths, rand=random.Random()):
    walks = []
    nodes = list(G.nodes())
    for i in range(num_paths):
        rand.shuffle(nodes)
        for node in nodes:
            walks.append(random_walk(G, start=node))
    return walks


# print(build_deepwalk_corpus(G, num_paths=2))


[['ONE', 'CREDIT', 'METHOD', 'CREDIT', 'Ringgit:', 'is', '�Whis', 'is', '60', '46', '23', '126', '126', '36,', '56', '249699', 'GEL', '7', '1S', '640102'], ['|NATIONALITY|', 'Pakistani', '|[NATIONALITY|', ':', '31', ':', 'Quantity', 'Price', 'Amoung]', 'I,', 'Zainab', 'Perempuan', 'Zainab', 'I,', '214.8', 'I,', '214.8', '37.90]', '3.7900)', '5.3700)'], ['452602', '/', 'INVOICE', '?', '?O', 'Cash', 'RU', '10029', 'AWFAO2', '10094', 'Ship-To', 'KLG', 'Location', 'BW', 'Ship-To', 'Vendor', 'Ship-To', 'KK', 'KBeonsave', 'Unit'], ['BAYARAN', 'SINGLE', 'BAYARAN', 'VISA', '222400', '984200', 'PROSES', 'ENTRY', 'YASIR', 'MEHMOOD', 'NAME', 'charged.', 'GST', 'THOUSAND', 'GST', 'Credit', 'by', 'Purchase', '40619-U', 'No:'], ['13.2', '1.32', '7.92', '1.32', '13.2', '26.4', '33.6', '26.4', '33.6', '1.68', '10', '115899', '115898', '9', '8', '115896', 'CLASSIC', 'PASTELS', 'LEAD', 'LEAD'], ['LOPC', 'Delivery', '1(C', 'YELLOW', 'ORANGE', 'UNKNOWN', '10X', 'BC', '/(142255)', 'SMM', '/(243502)', 'BC',

In [ ]:
from gensim.models import Word2Vec
corpus = build_deepwalk_corpus(G, num_paths=20)
model = Word2Vec(corpus, window=2, min_count=1, sg=1)


# Checking Word-Similarity

In [ ]:
model.wv.most_similar('Date:')

NameError: name 'model' is not defined

# Making Word-Embeddings

In [ ]:
_embeddings = []
all_Text = []

for i in range(100):
    df = pd.read_csv(lab_files_path+files[i])
    Text = df['Object'].to_list()
    
    for T in Text:
        all_Text.append(T)
    
print("Total text is: ", len(all_Text))
print("Making Embeddings..") 

Nodes = G.number_of_nodes()
# _embeddings = [model.wv[str(n)] for n in range(Nodes)]
# print(_embeddings)

Total text is:  19322
Making Embeddings..
